In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import requests
import pickle
import time
import bs4
import os

# 浏览器设置
options = Options()
options.headless = False  # 关闭无头模式，避免被检测
driver = webdriver.Chrome(options=options)

# Pixiv 主页
pixiv_url = "https://www.pixiv.net/"

# **1. 加载 cookies**
def load_cookies():
    if os.path.exists("cookies.pkl"):
        driver.get(pixiv_url)
        time.sleep(3)  # 等待页面加载
        with open("cookies.pkl", "rb") as f:
            cookies = pickle.load(f)
            for cookie in cookies:
                driver.add_cookie(cookie)
        print("✅ Cookies 加载成功！")
        driver.refresh()  # 刷新页面，使 cookies 生效
        time.sleep(5)
        return True
    return False

# **2. 登录并保存 cookies**
def login_and_save_cookies():
    driver.get("https://accounts.pixiv.net/login")
    print("🔑 请手动登录 Pixiv...")
    time.sleep(30)  # 给用户足够时间登录

    # 登录后，保存 cookies
    cookies = driver.get_cookies()
    print(cookies)
    with open("cookies.pkl", "wb") as f:
        pickle.dump(cookies, f)
    print("✅ 登录成功，Cookies 已保存！")

# **3. 主运行逻辑**
if not load_cookies():
    login_and_save_cookies()
else:
    print("✅ 已使用 Cookies 登录，无需手动输入密码！")

# **4. 爬取 Pixiv 页面**
driver.get(pixiv_url)
time.sleep(5)
print("🎉 已成功进入 Pixiv！")


✅ Cookies 加载成功！
✅ 已使用 Cookies 登录，无需手动输入密码！
🎉 已成功进入 Pixiv！


In [8]:
# 登录 Pixiv
page_links = []

def get_pixiv_series_images(url = "https://www.pixiv.net/user/14601125/series/217742?p=1#seriesContents"):
    try:
        driver.get(url)

        # 等待页面加载
        time.sleep(3)

        # 模拟滚动加载更多内容
        for _ in range(2):  # 根据需要调整滚动次数
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

        # 获取页面 HTML
        page_source = driver.page_source

        # 使用 BeautifulSoup 解析 HTML
        soup = bs4.BeautifulSoup(page_source, "html.parser")
        links = soup.find_all("a", class_="sc-d00cb9ca-0 jmvyWD sc-5a760b36-6 bsJslY gtm-gtm-manga-series-thumbnail")
        for link in links:
            href = link.get("href")
            if href not in page_links:
                page_links.append("https://www.pixiv.net" + href)
    except Exception as e:
        print(e)

for i in range(1, 5 + 1):
    url = f"https://www.pixiv.net/user/14601125/series/217742?p={i}#seriesContents"
    get_pixiv_series_images(url)
    print(f"已获取第 {i} 页的链接")
    time.sleep(2)

print("All links have been collected")

已获取第 1 页的链接
已获取第 2 页的链接
已获取第 3 页的链接
已获取第 4 页的链接
已获取第 5 页的链接
All links have been collected


In [14]:
def download_image(url, filename):
    headers = {
        "Referer": "https://www.pixiv.net/",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
    }

    session = requests.Session()
    for cookie in driver.get_cookies():
        session.cookies.set(cookie['name'], cookie['value'])
    
    response = session.get(url, headers=headers, stream=True) # stream=True 防止下载大文件时内存不够

    if response.status_code == 200:
        with open(filename, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):  # 默认的 chunk_size=8192，这里改为 1024
                f.write(chunk)
        print(f"✅ 下载完成: {filename}")
    else:
        print(f"❌ 下载失败: {url}, 状态码: {response.status_code}")

def get_image_links(url):
    print(url)
    driver.get(url)

    # 等待页面加载
    time.sleep(3)

    # 点击 "查看全部" 按钮
    """<div class="sc-emr523-2 wEKy">阅读作品 </div>
    //*[@id="root"]/div[2]/div/div[3]/div/div/div[1]/main/section/div[1]/div/div[5]/div/div/button/div[2]"""
    driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div/div[3]/div/div/div[1]/main/section/div[1]/div/div[5]/div/div/button/div[2]').click()

    # 模拟滚动加载更多内容
    for _ in range(2):  # 根据需要调整滚动次数
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    # 获取页面 HTML
    page_source = driver.page_source

    soup = bs4.BeautifulSoup(page_source, "html.parser")
    links = soup.find_all("a", class_="gtm-expand-full-size-illust")
    for i, link in enumerate(links):
        href = link.get("href")
        filename = link.get("href").split("/")[-1]
        download_image(href, filename)
        print(f"已经下载{i + 1}/{len(links)}张图片")
        time.sleep(1)

    print("All images have been downloaded")

for link in page_links:
    get_image_links(link)


https://www.pixiv.net/artworks/123828051


InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF67F054C25+3179557]
	(No symbol) [0x00007FF67ECB88A0]
	(No symbol) [0x00007FF67EB48FFC]
	(No symbol) [0x00007FF67EB8F8DF]
	(No symbol) [0x00007FF67EBC7AF2]
	(No symbol) [0x00007FF67EBC244E]
	(No symbol) [0x00007FF67EBC14F9]
	(No symbol) [0x00007FF67EB155E5]
	GetHandleVerifier [0x00007FF67F0B67CD+3579853]
	GetHandleVerifier [0x00007FF67F0CD1D2+3672530]
	GetHandleVerifier [0x00007FF67F0C2153+3627347]
	GetHandleVerifier [0x00007FF67EE2092A+868650]
	(No symbol) [0x00007FF67ECC2FFF]
	(No symbol) [0x00007FF67EB141FF]
	GetHandleVerifier [0x00007FF67F13FA28+4141608]
	BaseThreadInitThunk [0x00007FFFD47EE8D7+23]
	RtlUserThreadStart [0x00007FFFD5D5BF2C+44]
